# Installation of required packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/GeSumGenEval/src

/content/drive/.shortcut-targets-by-id/1PcWXs_So5sTaP0wBAR77_ORSfd4aFtHq/GeSumGenEval/src


In [3]:
!pip install -r ../requirements.txt

Processing /content/drive/.shortcut-targets-by-id/1PcWXs_So5sTaP0wBAR77_ORSfd4aFtHq/GeSumGenEval/emnlp19-moverscore
  Created wheel for moverscore: filename=moverscore-0.96-cp37-none-any.whl size=8826 sha256=b2e35c828be912afff30c4d5b922a83baccdc5fd56788e884212cbb800997133
  Stored in directory: /root/.cache/pip/wheels/79/75/ea/13343d846ad4741f6c07088b71b7c8038a182240999ca30895
Successfully built moverscore
  Found existing installation: moverscore 0.96
    Uninstalling moverscore-0.96:
      Successfully uninstalled moverscore-0.96


In [4]:
# For using pyrouge
!chmod 755 -R ROUGE-1.5.5/
# install missing dependency
!apt install libxml-parser-perl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libxml-parser-perl is already the newest version (2.44-2build3).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jun 30 14:00:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# MLSUM

In [ ]:
!python train.py -mode prune -bert_data_path ../bert_data/mlsum -test_from ../models/mlsum/model_step_45000.pt -log_file ../logs/mlsum -result_path ../json_data/mlsum -visible_gpus 0

In [8]:
#!python prepro/get_candidate.py --tokenizer=bert --data_path=../json_data/mlsum/train_original.jsonl --index_path=../json_data/mlsum/train_index.jsonl --write_path=../bert_data/mlsum/train.jsonl
#!python prepro/get_candidate.py --tokenizer=bert --data_path=../json_data/mlsum/valid_original.jsonl --index_path=../json_data/mlsum/valid_index.jsonl --write_path=../bert_data/mlsum/valid.jsonl
!python prepro/get_candidate.py --tokenizer=bert --data_path=../json_data/mlsum/test_original.jsonl --index_path=../json_data/mlsum/test_index.jsonl --write_path=../bert_data/mlsum/test.jsonl

Downloading: 100% 240k/240k [00:00<00:00, 6.38MB/s]
Downloading: 100% 59.0/59.0 [00:00<00:00, 84.9kB/s]
total 10701 documents
start getting candidates with multi-processing !!!
10701it [05:02, 35.41it/s]
finished in 0:05:02.364185
start writing 10701 files


In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_matching.py --mode=train --encoder=bert --save_path=../models/mlsum --gpus=0 --batch_size=2 --valid_steps=5000 --train_from=../models/mlsum/matchsum_last_saved_checkpoint.pt

In [11]:
!CUDA_VISIBLE_DEVICES=0 python train_matching.py --mode=test --encoder=bert --save_path=../results/mlsum --gpus=0 --test_from=../models/mlsum/epoch-2_step-190000_ROUGE-0.350548.pt

Testing process of MatchSum !!!
Start loading datasets !!!
Finished in 0:00:03.663228
Information of dataset is:
In total 1 datasets:
	test has 10701 instances.

Current model is ../models/mlsum/epoch-2_step-190000_ROUGE-0.350548.pt
10701/10701 (100.00%) decoded in 0:18:31 seconds
Start writing files !!!
Evaluate data in 1109.73 seconds!
[tester] 
MatchRougeMetric: 


# GeWiki

In [ ]:
import pandas as pd
import nltk
import plotly.express as px
import sys

nltk.download('punkt')

def get_word_len(source: str, language='german') -> int:
    words = nltk.sent_tokenize(source, language)

    return len(words)

# generating summaries only for the test set
gewiki_dataset = pd.read_csv("../data/gewiki/valid.csv")
gewiki_dataset['text_word_len'], gewiki_dataset['sum_word_len'] = map(list,zip(*[(
                                                      get_word_len(row['text']), 
                                                      get_word_len(row['summary']) 
                                                    ) for index, row in gewiki_dataset.iterrows()]))



fig1 = px.histogram(gewiki_dataset, x="text_word_len", labels={'text_word_len':'No. of words in source article'})
fig1.show()

fig2 = px.histogram(gewiki_dataset, x="sum_word_len", labels={'sum_word_len':'No. of words in expert summary'})
fig2.show()

In [12]:
!python train.py -mode prune -bert_data_path ../bert_data/gewiki -test_from ../models/gewiki/model_step_40000.pt -log_file ../logs/gewiki -result_path ../json_data/gewiki -visible_gpus 0

[2021-06-30 14:44:56,216 INFO] Loading checkpoint from ../models/gewiki/model_step_40000.pt
Namespace(accum_count=1, batch_size=1000, bert_config_path='../bert_config_uncased_base.json', bert_data_path='../bert_data/gewiki', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='transformer', ff_size=2048, gpu_ranks=[0], heads=8, hidden_size=128, inter_layers=2, log_file='../logs/gewiki', lr=1, max_grad_norm=0, mode='prune', model_path='models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=False, result_path='../json_data/gewiki', rnn_size=512, save_checkpoint_steps=5, seed=666, temp_dir='temp', test_all=False, test_from='../models/gewiki/model_step_40000.pt', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_steps=8000, world_size=1)
[2021-06-30 14:45:15,595 INFO] Loading test dataset from ../bert_data/gewiki/test.0.bert.pt, number of examples: 2000
gpu_rank 0
[20

In [13]:
#!python prepro/get_candidate.py --tokenizer=bert --data_path=../json_data/gewiki/train_original.jsonl --index_path=../json_data/gewiki/train_index.jsonl --write_path=../bert_data/gewiki/train.jsonl
#!python prepro/get_candidate.py --tokenizer=bert --data_path=../json_data/gewiki/valid_original.jsonl --index_path=../json_data/gewiki/valid_index.jsonl --write_path=../bert_data/gewiki/valid.jsonl
!python prepro/get_candidate.py --tokenizer=bert --data_path=../json_data/gewiki/test_original.jsonl --index_path=../json_data/gewiki/test_index.jsonl --write_path=../bert_data/gewiki/test.jsonl

total 10000 documents
start getting candidates with multi-processing !!!
10000it [05:48, 28.71it/s]
finished in 0:05:48.481400
start writing 10000 files


In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_matching.py --mode=train --encoder=bert --save_path=../models/gewiki --gpus=0 --batch_size=2 --valid_steps=5000 --train_from=../models/gewiki/matchsum_last_saved_checkpoint.pt

In [14]:
!CUDA_VISIBLE_DEVICES=0 python train_matching.py --mode=test --encoder=bert --save_path=../results/gewiki --gpus=0 --test_from=../models/gewiki/epoch-2_step-270000_ROUGE-0.173690.pt

Testing process of MatchSum !!!
Start loading datasets !!!
Finished in 0:00:03.962841
Information of dataset is:
In total 1 datasets:
	test has 10000 instances.

Current model is ../models/gewiki/epoch-2_step-270000_ROUGE-0.173690.pt
10000/10000 (100.00%) decoded in 0:19:38 seconds
Start writing files !!!
Evaluate data in 1176.86 seconds!
[tester] 
MatchRougeMetric: 
